<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Deepfake_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### **Deepfake and Manipulated Media Analysis Data Download**

In [1]:
!pip install -qU yt-dlp aiohttp decord pandas pillow soundfile tqdm crawl4ai[all] nest_asyncio playwright

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.0/172.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 70.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.6/13.6 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.1/45.1 MB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.1/201.1 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.5/287.5 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/1

In [2]:
import nest_asyncio
nest_asyncio.apply()

import os
import asyncio
import aiohttp
import json
import subprocess
from datetime import datetime

import pandas as pd
import yt_dlp
from tqdm.asyncio import tqdm
from playwright.async_api import async_playwright

# Import crawl4ai components
from crawl4ai import AsyncWebCrawler
from crawl4ai.extraction_strategy import JsonCssExtractionStrategy

#########################################
# Setup Directories and Global Variables
#########################################
BASE_DIR = os.getcwd()
SCRAPED_URLS_DIR = os.path.join(BASE_DIR, 'scraped_urls')
IMAGES_DIR      = os.path.join(BASE_DIR, 'deepfake_images')
VIDEOS_DIR      = os.path.join(BASE_DIR, 'deepfake_videos')
AUDIOS_DIR      = os.path.join(BASE_DIR, 'deepfake_audios')
METADATA_FILE   = os.path.join(BASE_DIR, 'download_metadata.json')

for d in [SCRAPED_URLS_DIR, IMAGES_DIR, VIDEOS_DIR, AUDIOS_DIR]:
    os.makedirs(d, exist_ok=True)

# Global list to hold download metadata
download_metadata = []

#########################################
# Helper: Ensure Playwright Browsers Are Installed
#########################################
def install_playwright_browsers():
    try:
        print("[SYSTEM] Installing Playwright browsers...")
        subprocess.run(["playwright", "install"], check=True)
        print("[SYSTEM] Playwright browsers installed.")
    except Exception as e:
        print(f"[SYSTEM] Error installing Playwright browsers: {e}")

In [3]:
#########################################
# 1. Define URL Scraping Functions Using crawl4ai
#########################################
async def scrape_media_urls(modality, page_url, parse_function, output_filename):
    """
    Scrape media URLs from a given page URL using crawl4ai's AsyncWebCrawler and a CSS extraction strategy.
    Saves the scraped URLs as a CSV file and returns a list of unique URLs.
    """
    print(f"[{modality.upper()}] Scraping URLs from {page_url}")

    # Define a simple extraction schema.
    schema = {"extracted": list}

    extraction_strategy = JsonCssExtractionStrategy(
        parse_function=parse_function,
        schema=schema
    )

    crawler = AsyncWebCrawler(start_urls=[page_url], extraction_strategy=extraction_strategy)

    try:
        # Use the new method `start()` (adjust if your crawl4ai version differs)
        results = await crawler.start()
    except Exception as e:
        print(f"[{modality.upper()}] Error during crawling {page_url}: {e}")
        results = []

    scraped_urls = []
    for result in results:
        scraped_urls.extend(result.get('extracted', []))

    # Remove duplicates and empty values
    scraped_urls = list({url for url in scraped_urls if url})

    # Save URLs to CSV for future reference
    df = pd.DataFrame({'url': scraped_urls})
    df.to_csv(output_filename, index=False)
    print(f"[{modality.upper()}] Saved {len(scraped_urls)} URLs to {output_filename}")
    return scraped_urls

# Define parse functions (adjust CSS selectors to match your target deepfake pages)
def parse_image_links(response):
    # Example: extract src attribute from images with class "deepfake-img"
    return response.css("img::attr(src)").getall()

def parse_video_links(response):
    # Example: extract href attribute from anchors with class "deepfake-video"
    return response.css("a::attr(href)").getall()

def parse_audio_links(response):
    # Example: extract src attribute from audio tags with class "deepfake-audio"
    return response.css("audio::attr(src)").getall()

In [4]:
#########################################
# 2. Define Functions to Load URLs (with Fallbacks)
#########################################
def load_scraped_urls(modality):
    filepath = os.path.join(SCRAPED_URLS_DIR, f"{modality}_urls.csv")
    if os.path.exists(filepath):
        df = pd.read_csv(filepath)
        urls = df['url'].dropna().tolist()
        print(f"[{modality.upper()}] Loaded {len(urls)} URLs from {filepath}")
        return urls
    else:
        print(f"[{modality.upper()}] File {filepath} not found. Using fallback method.")
        return []

# Fallback generators for deepfake-specific media
def fallback_image_urls(n=20):
    # Create diverse deepfake-specific image URLs by seeding with 'deepfake'
    return [f"https://picsum.photos/seed/deepfake{i}/600/400" for i in range(n)]

def fallback_video_urls(n=20):
    # Leave empty to trigger yt-dlp's search fallback, which uses a deepfake compilation query.
    return []

def fallback_audio_urls(n=20):
    # Leave empty so that we attempt extraction using Playwright fallback.
    return []

In [5]:
#########################################
# 3. Define Asynchronous Download Functions with Retries & Metadata Logging
#########################################
async def download_file(session, url, filename, modality, retries=3):
    """
    Download a file (image or audio) asynchronously using aiohttp with retry logic.
    Saves the file and logs metadata.
    """
    for attempt in range(1, retries + 1):
        try:
            async with session.get(url) as resp:
                if resp.status == 200:
                    content = await resp.read()
                    with open(filename, "wb") as f:
                        f.write(content)
                    print(f"[{modality.upper()}] Downloaded: {filename}")
                    download_metadata.append({
                        "modality": modality,
                        "filename": filename,
                        "url": url,
                        "download_time": datetime.now().isoformat()
                    })
                    return
                else:
                    print(f"[{modality.upper()}] Attempt {attempt}: Failed to download {url} (Status {resp.status})")
        except Exception as e:
            print(f"[{modality.upper()}] Attempt {attempt}: Exception downloading {url}: {e}")
        await asyncio.sleep(1)
    print(f"[{modality.upper()}] Failed to download {url} after {retries} attempts.")

async def download_images(urls):
    print("[IMAGE] Starting asynchronous image downloads...")
    async with aiohttp.ClientSession() as session:
        tasks = []
        for idx, url in enumerate(urls):
            filename = os.path.join(IMAGES_DIR, f"deepfake_image_{idx+1:02d}.jpg")
            tasks.append(download_file(session, url, filename, modality="image"))
        await asyncio.gather(*tasks)

async def download_audios(urls):
    if not urls:
        print("[AUDIO] No audio URLs provided for download.")
        return
    print("[AUDIO] Starting asynchronous audio downloads...")
    async with aiohttp.ClientSession() as session:
        tasks = []
        for idx, url in enumerate(urls):
            filename = os.path.join(AUDIOS_DIR, f"deepfake_audio_{idx+1:02d}.mp3")
            tasks.append(download_file(session, url, filename, modality="audio"))
        await asyncio.gather(*tasks)

In [6]:
#########################################
# 4. Define Synchronous Video Download Function Using yt-dlp
#########################################
def download_videos(video_urls, num_videos=20):
    """
    Download videos using yt-dlp. If no scraped URLs exist, use yt-dlp search fallback.
    The fallback query is tuned to deepfake compilations.
    """
    if video_urls:
        download_list = video_urls[:num_videos]
        print("[VIDEO] Downloading videos from scraped URLs...")
        for url in download_list:
            ydl_opts = {
                'format': 'bestvideo+bestaudio/best',
                'outtmpl': os.path.join(VIDEOS_DIR, '%(id)s.%(ext)s'),
                'merge_output_format': 'mp4',
                'quiet': True,
                'no_warnings': True,
            }
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                try:
                    ydl.download([url])
                    download_metadata.append({
                        "modality": "video",
                        "filename": url,  # You may wish to parse actual filename later.
                        "url": url,
                        "download_time": datetime.now().isoformat()
                    })
                    print(f"[VIDEO] Downloaded: {url}")
                except Exception as e:
                    print(f"[VIDEO] Error downloading {url}: {e}")
    else:
        # Use yt-dlp fallback search for deepfake compilations
        search_query = "ytsearch20:deepfake compilation"
        ydl_opts = {
            'format': 'bestvideo+bestaudio/best',
            'outtmpl': os.path.join(VIDEOS_DIR, '%(id)s.%(ext)s'),
            'merge_output_format': 'mp4',
            'quiet': True,
            'no_warnings': True,
        }
        print("[VIDEO] No scraped video URLs found; using yt-dlp search fallback...")
        with yt_dlp.YoutubeDL(ydl_opts) as ydl:
            try:
                ydl.download([search_query])
                print("[VIDEO] Video downloads (fallback search) complete.")
            except Exception as e:
                print(f"[VIDEO] Error with yt-dlp search fallback: {e}")

In [7]:
#########################################
# 5. Define Audio Extraction Using Playwright as a Fallback
#########################################
async def extract_audio_links():
    """
    Use Playwright to extract deepfake-specific audio URLs from a dynamic page.
    Update the URL and CSS selectors to match a real deepfake audio source.
    """
    print("[AUDIO] Extracting audio URLs via Playwright fallback...")
    extracted_links = []
    try:
        async with async_playwright() as p:
            browser = await p.chromium.launch(headless=True)
            context = await browser.new_context()
            page = await context.new_page()
            # Update this URL to a site known for synthetic or deepfake audio samples
            url = "https://uberduck.ai/explore"  # Example URL; may need replacement
            await page.goto(url)
            await page.wait_for_timeout(5000)
            # Adjust the selector based on the site’s structure
            audio_elements = await page.query_selector_all("audio")
            for el in audio_elements:
                src = await el.get_attribute("src")
                if src:
                    extracted_links.append(src)
            await browser.close()
            print(f"[AUDIO] Extracted {len(extracted_links)} audio URLs via Playwright.")
    except Exception as e:
        if "Executable doesn't exist" in str(e):
            install_playwright_browsers()
        print(f"[AUDIO] Error during Playwright audio extraction: {e}")
    return extracted_links

In [8]:
#########################################
# 6. Save Download Metadata to a JSON File
#########################################
def save_download_metadata(metadata_list, filename=METADATA_FILE):
    try:
        with open(filename, 'w') as f:
            json.dump(metadata_list, f, indent=4)
        print(f"[SYSTEM] Download metadata saved to {filename}")
    except Exception as e:
        print(f"[SYSTEM] Error saving download metadata: {e}")

In [9]:
#########################################
# 7. Main Pipeline
#########################################
async def main_pipeline():
    # --- Configuration: Define the source pages for each modality ---
    pages = {
        "face_swapping": "https://thispersondoesnotexist.com/",
        "face_forgery": "https://www.whichfaceisreal.com/",
        "deepfake_voice_detection": "https://www.kaggle.com/datasets/search?query=deepfake+voice",
        "speaker_verification_spoofing": "https://www.kaggle.com/datasets/search?query=speaker+verification+spoofing",
        "audio_tampering_splicing": "https://www.kaggle.com/datasets/search?query=audio+tampering",
        "replay_attacks": "https://www.kaggle.com/datasets/search?query=replay+attacks",
        "multi_modal_fake_detection": "https://www.kaggle.com/datasets/search?query=multi+modal+fake+detection",
        "deepfake_video_detection": "https://www.kaggle.com/datasets/search?query=deepfake+video+detection",
        "general_video_manipulation": "https://www.kaggle.com/datasets/search?query=video+manipulation",
        "image_splicing_copy_move": "https://www.kaggle.com/datasets/search?query=image+splicing",
        "ai_generated_image_manipulations": "https://www.kaggle.com/datasets/search?query=ai+generated+images",
        "large_scale_photoshop_edits": "https://www.kaggle.com/datasets/search?query=photoshop+edits"
    }

    # --- Scrape URLs Using crawl4ai (if pages are accessible) ---
    for modality, page_url in pages.items():
        if "image" in modality:
            urls = await scrape_media_urls(modality, page_url, parse_image_links,
                                           os.path.join(SCRAPED_URLS_DIR, f"{modality}_urls.csv"))
        elif "video" in modality:
            urls = await scrape_media_urls(modality, page_url, parse_video_links,
                                           os.path.join(SCRAPED_URLS_DIR, f"{modality}_urls.csv"))
        elif "audio" in modality:
            urls = await scrape_media_urls(modality, page_url, parse_audio_links,
                                           os.path.join(SCRAPED_URLS_DIR, f"{modality}_urls.csv"))
        else:
            urls = []

        # --- Load URLs (if available); otherwise, use fallbacks ---
        if not urls:
            print(f"[{modality.upper()}] No scraped URLs; using fallback.")
            if "image" in modality:
                urls = fallback_image_urls(20)
            elif "video" in modality:
                urls = fallback_video_urls(20)
            elif "audio" in modality:
                urls = fallback_audio_urls(20)
        else:
            urls = urls[:20]

        # --- Download Media Asynchronously ---
        if "image" in modality:
            await download_images(urls)
        elif "video" in modality:
            download_videos(urls, num_videos=20)
        elif "audio" in modality:
            await download_audios(urls)

    # --- Save Download Metadata ---
    save_download_metadata(download_metadata)
    print("[SYSTEM] All downloads complete. Check the respective directories for files.")

# Run the entire pipeline.
if __name__ == '__main__':
    asyncio.run(main_pipeline())

[FACE_SWAPPING] No scraped URLs; using fallback.
[FACE_FORGERY] No scraped URLs; using fallback.
[DEEPFAKE_VOICE_DETECTION] No scraped URLs; using fallback.
[SPEAKER_VERIFICATION_SPOOFING] No scraped URLs; using fallback.
[AUDIO_TAMPERING_SPLICING] Scraping URLs from https://www.kaggle.com/datasets/search?query=audio+tampering
[AUDIO_TAMPERING_SPLICING] Error during crawling https://www.kaggle.com/datasets/search?query=audio+tampering: BrowserType.launch: Executable doesn't exist at /root/.cache/ms-playwright/chromium-1155/chrome-linux/chrome
╔════════════════════════════════════════════════════════════╗
║ Looks like Playwright was just installed or updated.       ║
║ Please run the following command to download new browsers: ║
║                                                            ║
║     playwright install                                     ║
║                                                            ║
║ <3 Playwright Team                                         ║
╚═════════